In [1]:
import csv
import re
import pandas as pd
import time

import selenium 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.chrome.options import Options

In [2]:
sample_df = pd.read_csv("PUBCHEM_SYNONYMS_FLAGGED.csv")
# sample_df = sample_df.head(60)
cas_df = sample_df[sample_df["CAS"] != "No CAS info"]
missing_df = sample_df[sample_df["CAS"] == "No CAS info"] 

In [3]:
cas_df = cas_df[['Parsed Molecule', 'CAS']]
missing_df = missing_df[['Parsed Molecule', 'CAS']]

In [4]:
def get_first_cas(cas_string):
    # Replace all semicolons with commas
    modified_string = re.sub(r';', ',', cas_string)
    
    if modified_string == "No CAS info":
        return None
    
    return modified_string.split(',')[0].strip()

# Apply this function to the entire CAS column
cas_df['CAS'] = cas_df['CAS'].apply(get_first_cas)
cas_arr = cas_df['CAS'].tolist()

missing_df['CAS'] = missing_df['CAS'].apply(get_first_cas)
missing_arr = missing_df['CAS'].tolist()

In [5]:
def setup_webdriver():
    '''Initializes a headless selenium webdriver'''
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def url_inxight(cas):
    '''Generates an Inxight URL for a given CAS'''
    return f"https://drugs.ncats.io/substances?q={cas}"

# def url_inxight(cas):
#     '''Generates an Inxight URL for a given CAS'''
    return f"https://drugs.ncats.io/substances?q=%22{cas}%22&facet=Substance%20Form%2FPrincipal%20Form"

In [6]:
def get_inxight_url(molecule, driver):
    '''
    Parameters
    ----------
    A molecule's CAS and an initialised webdriver.

    Returns
    -------
    The top Inxight search result for a given CAS number
    '''
    
    url = "N/A"  # Default in case of failure

    driver.get(url_inxight(molecule))
    try:
        WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'a[id="card-title"]')))
        elements = driver.find_elements(By.CSS_SELECTOR, 'a[id="card-title"]')
        if elements:
            element = elements[0]
            url = element.get_attribute('href')

    except (NoSuchElementException, TimeoutException):
        pass  

    return molecule, url

def get_best_urls(cas_arr):
    '''
    Parameters
    ----------
    An array of CAS values

    Returns
    -------
    An array of associated top matches
    '''
    
    best_matches = []
    driver = setup_webdriver()  
    try:
        for cas in cas_arr:
            molecule, url = get_inxight_url(cas, driver)
            best_matches.append((molecule, url))
    finally:
        driver.quit()  

    return best_matches

In [7]:
links = get_best_urls(cas_arr)

In [8]:
links

[('623-84-7', 'https://drugs.ncats.io/drug/5Z492UNF9O'),
 ('36653-82-4', 'https://drugs.ncats.io/drug/936JST6JCN'),
 ('112-92-5', 'https://drugs.ncats.io/drug/2KR89I4H1Y'),
 ('328-50-7', 'https://drugs.ncats.io/drug/8ID597Z82X'),
 ('67-63-0', 'https://drugs.ncats.io/drug/ND2M416302'),
 ('56-12-2', 'https://drugs.ncats.io/drug/2ACZ6IPC6I'),
 ('566-19-8', 'https://drugs.ncats.io/drug/2334LJD2E9'),
 ('148-24-3', 'https://drugs.ncats.io/drug/5UTX5635HP'),
 ('136470-78-5', 'https://drugs.ncats.io/drug/WR2TIP26VS'),
 ('247062-33-5', 'https://drugs.ncats.io/drug/AVK0I6HY2U'),
 ('183552-38-7', 'https://drugs.ncats.io/drug/W486SJ5824'),
 ('332348-12-6', 'https://drugs.ncats.io/drug/7D0YB67S97'),
 ('143653-53-6', 'https://drugs.ncats.io/drug/X85G7936GV'),
 ('1231929-97-7', 'https://drugs.ncats.io/drug/60UAB198HK'),
 ('154229-18-2', 'https://drugs.ncats.io/drug/EM5OCB9YJ6'),
 ('1420477-60-6', 'https://drugs.ncats.io/drug/I42748ELQW'),
 ('77337-76-9', 'https://drugs.ncats.io/drug/N4K14YGM3J'),
 ('

In [9]:
len(links)

2321

In [10]:
identifiers = [url.split('/')[-1] for _, url in links]
identifiers = [identifier if identifier != 'A' else 'MISSING' for identifier in identifiers]
identifiers

['5Z492UNF9O',
 '936JST6JCN',
 '2KR89I4H1Y',
 '8ID597Z82X',
 'ND2M416302',
 '2ACZ6IPC6I',
 '2334LJD2E9',
 '5UTX5635HP',
 'WR2TIP26VS',
 'AVK0I6HY2U',
 'W486SJ5824',
 '7D0YB67S97',
 'X85G7936GV',
 '60UAB198HK',
 'EM5OCB9YJ6',
 'I42748ELQW',
 'N4K14YGM3J',
 'T58MSI464G',
 '67P356D8GH',
 'UZ29E6L2X8',
 '54EJ303F0R',
 'MA3UYZ6K1H',
 'O3FX965V0I',
 'Q40Q9N063P',
 'QGC8W08I6I',
 '4RZ82L2GY5',
 '1364PS73AF',
 '6DH1W9VH8Q',
 'N9YNS0M02X',
 'WYQ7N0BPYC',
 'R16CO5Y76E',
 'WBT5QH3KED',
 '80VUN7L00C',
 'X4HES1O11F',
 'LCH760E9T7',
 'UQW7UF9N91',
 'A20F9XAI7W',
 'ad622025',
 '1L4806J2QF',
 'U6Q8Z01514',
 '7LP2MPO46S',
 'K72T3FS567',
 '8L70Q75FXE',
 'TKQ858A3VW',
 '41UD74L59M',
 '1567f7b0',
 '2HLC17MX9G',
 'OF5P57N2ZX',
 '7QVV6I50DT',
 'F4216019LN',
 '5E7U48495E',
 'P88XT4IS4D',
 'A57KX1VL5P',
 '7Z8O94ECSX',
 '136H45TB7B',
 'M89N0Q7EQR',
 '8T66I31YNK',
 'LIJ4CT1Z3Y',
 'ELK3V90G6C',
 '3A189DH42V',
 'X1J18R4W8P',
 '1N74HM2BS7',
 '90347YTW5F',
 '8C3Z4148WZ',
 '27T56C7KK0',
 'DTI67O9503',
 'PP0SHH6V16',

In [25]:
# def extract_conditions_and_phases(data):
#     conditions_list = []
#     highest_approval_list = []

#     for entry in data:
#         if entry['name'] == 'Conditions' and 'value' in entry:
#             condition_info = entry['value']
            
#             # Extract the condition name
#             label = condition_info.get('label')
#             if label:
#                 conditions_list.append(label)
            
#             # Extract the highest phase of approval
#             highest_phase = condition_info.get('highestPhase')
#             if highest_phase:
#                 highest_approval_list.append(highest_phase)

#     return conditions_list, highest_approval_list

def extract_conditions_and_phases(data):
    """Extract conditions and their highest phases of approval from the API data."""
    conditions_list = []
    highest_approval_list = []

    for entry in data:
        if isinstance(entry, dict) and entry.get('name') == 'Conditions' and 'value' in entry:
            condition_info = entry['value']
            
            # Extract the condition name
            label = condition_info.get('label')
            if label:
                conditions_list.append(label)
            
            # Extract the highest phase of approval
            highest_phase = condition_info.get('highestPhase')
            if highest_phase:
                highest_approval_list.append(highest_phase)

    return conditions_list, highest_approval_list

In [12]:
def extract_event_details(data):
    event_details = {}
    
    for item in data:
        if 'value' in item and isinstance(item['value'], dict):  # Ensure 'value' is a dictionary
            details = item['value']
            if 'status' in details and 'sourceID' in details:
                if item['name'] == 'Highest Development Event' or item['name'] == 'Earliest Approved Event':
                    # Gather additional details
                    source_id = details.get('sourceID', 'No Source ID')
                    source_url = details.get('sourceURL', 'No Source URL')
                    
                    detail_info = {
                        'Status and Year': f"{details['status']} {details.get('year', '')}",
                        'Source ID': source_id,
                        'Source URL': source_url
                    }
                    
                    # Use the 'name' of the event as the key in the dictionary
                    event_details[item['name']] = detail_info
    
    return event_details

In [13]:
# def extract_events(identifiers):
#     events = []

#     for identifier in identifiers:
#         if identifier == "MISSING":
#             events.append({'Highest Development Event': {'Status and Year': 'Possibly Marketed Outside US 1996',
#    'Source ID': 'ANDA040166',
#    'Source URL': 'https://dailymed.nlm.nih.gov/dailymed/drugInfo.cfm?setid=c0ec47b7-c13a-4fa0-91fe-d7a03c70aec9'},
#   'Earliest Approved Event': {'Status and Year': 'Possibly Marketed Outside US 1996',
#    'Source ID': 'ANDA040166',
#    'Source URL': 'https://dailymed.nlm.nih.gov/dailymed/drugInfo.cfm?setid=c0ec47b7-c13a-4fa0-91fe-d7a03c70aec9'}})
#         data = get_additional_data(identifier)
#         print(extract_event_details(data))
#         events.append(extract_event_details(data))

#     return events
        
# events_arr = extract_events(identifiers)

def extract_events(identifiers):
    events = []
    acc = 0

    for identifier in identifiers:
        acc += 1
        print(acc)
        if identifier == "MISSING":
            # Create a dictionary where all details are 'MISSING'
            missing_details = {
                'Highest Development Event': {
                    'Status and Year': 'N/A',
                    'Source ID': 'N/A',
                    'Source URL': 'N/A'
                },
                'Earliest Approved Event': {
                    'Status and Year': 'N/A',
                    'Source ID': 'N/A',
                    'Source URL': 'N/A'
                }
            }
            events.append(missing_details)
        else:
            data = get_additional_data(identifier)
            event_details = extract_event_details(data)
            events.append(event_details)

    return events

In [26]:
def extract_conditions(identifiers):
    """Extract conditions for a list of identifiers."""
    conditions = []

    for identifier in identifiers:
        if identifier == "MISSING":
            conditions.append((['N/A'], ['N/A']))
        else:
            data = get_additional_data(identifier)
            if data:  # Check if data is not empty
                extracted_data = extract_conditions_and_phases(data)
                conditions.append(extracted_data)
            else:
                conditions.append(([], []))  # Append empty lists if no data
    return conditions

In [19]:
import requests

def get_additional_data(id):
    r = requests.get(f'https://drugs.ncats.io/api/v1/substances({id})/@additional')
    if 200 == r.status_code:
        return r.json()
    return '{}'

In [20]:
events_arr = extract_events(identifiers)
events_arr

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


[{'Highest Development Event': {'Status and Year': 'Possibly Marketed Outside US 1996',
   'Source ID': 'ANDA040166',
   'Source URL': 'https://dailymed.nlm.nih.gov/dailymed/drugInfo.cfm?setid=c0ec47b7-c13a-4fa0-91fe-d7a03c70aec9'},
  'Earliest Approved Event': {'Status and Year': 'Possibly Marketed Outside US 1996',
   'Source ID': 'ANDA040166',
   'Source URL': 'https://dailymed.nlm.nih.gov/dailymed/drugInfo.cfm?setid=c0ec47b7-c13a-4fa0-91fe-d7a03c70aec9'}},
 {'Highest Development Event': {'Status and Year': 'US Previously Marketed 1990',
   'Source ID': 'EXOSURF NEONATAL by GLAXOSMITHKLINE',
   'Source URL': 'https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=020044'},
  'Earliest Approved Event': {'Status and Year': 'Possibly Marketed Outside US 1984',
   'Source ID': 'NU-DERM SUNFADER  Skin Lightener with Sunscreen (SPF 15) PABA FREE by OMP, INC.',
   'Source URL': 'https://dailymed.nlm.nih.gov/dailymed/drugInfo.cfm?setid=c67b0629-5529-412d-99e

In [27]:
import requests

def get_additional_data(id):
    r = requests.get(f'https://drugs.ncats.io/api/v1/substances({id})/@additional')
    if 200 == r.status_code:
        return r.json()
    return ([], [])

In [28]:
conditions_arr = extract_conditions(identifiers)
conditions_arr

[(['Bacillus cereus infection'], ['Basic research']),
 (['Dry, itchy skin', 'Respiratory distress syndrome, neonatal'],
  ['Approved', 'Approved']),
 (['SUNBURN'], ['Approved']),
 (['Osteoporosis',
   'Gastrointestinal microbiota imbalance',
   'Short bowel syndrome',
   'Burn',
   'Kidney failure'],
  ['Not Provided', 'Not Provided', 'Not Provided', 'Approved', 'Approved']),
 ([], []),
 (['Type 1 diabetes mellitus',
   'Stress',
   'Vascular diseases of the brain',
   'Atherosclerosis of cerebral arteries',
   'Chronic cerebrovascular insufficiency',
   'Partial Seizures'],
  ['Phase II', 'Phase I', 'Approved', 'Approved', 'Approved', 'Not Provided']),
 ([], []),
 (['Unknown'], []),
 (['HIV-1 infection'], ['Approved']),
 (['Osteoporosis'], ['Approved']),
 (['Advanced prostate cancer'], ['Approved']),
 ([], []),
 ([], []),
 (['Breast cancer',
   'Melanoma',
   'Glioblastoma',
   'Non-small cell lung cancer',
   'HER2-negative advanced breast cancer'],
  ['Phase III', 'Phase II', 'Phase

In [29]:
data = []
for event in events_arr:
    record = {}
    for key, value in event.items():
        for sub_key, sub_value in value.items():
            record[f"{key} {sub_key}"] = sub_value
    data.append(record)

df = pd.DataFrame(data)

In [30]:
rows = [{'conditions': conditions, 'phases': phases} for conditions, phases in conditions_arr]

# Create DataFrame
conditions_df = pd.DataFrame(rows)
conditions_df['conditions'] = conditions_df['conditions'].apply(lambda x: '; '.join(x))
conditions_df['phases'] = conditions_df['phases'].apply(lambda x: '; '.join(x))
conditions_df

,conditions,phases
0,Bacillus cereus infection,Basic research
1,"Dry, itchy skin; Respiratory distress syndrome...",Approved; Approved
2,SUNBURN,Approved
3,Osteoporosis; Gastrointestinal microbiota imba...,Not Provided; Not Provided; Not Provided; Appr...
4,,
...,...,...
2316,,
2317,Postmenopausal osteoporosis; Hypercalcemia; Pr...,Approved; Approved; Approved; Approved; Approved
2318,Migraine with aura; Migraine without aura,Approved; Approved
2319,Insomnia,Approved


In [31]:
merged = pd.concat([df, conditions_df], axis=1)
merged

,Highest Development Event Status and Year,Highest Development Event Source ID,Highest Development Event Source URL,Earliest Approved Event Status and Year,Earliest Approved Event Source ID,Earliest Approved Event Source URL,conditions,phases
0,Possibly Marketed Outside US 1996,ANDA040166,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,Possibly Marketed Outside US 1996,ANDA040166,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,Bacillus cereus infection,Basic research
1,US Previously Marketed 1990,EXOSURF NEONATAL by GLAXOSMITHKLINE,https://www.accessdata.fda.gov/scripts/cder/da...,Possibly Marketed Outside US 1984,NU-DERM SUNFADER Skin Lightener with Sunscree...,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,"Dry, itchy skin; Respiratory distress syndrome...",Approved; Approved
2,US Previously Marketed,21 CFR 310.545(a)(18)(i)(B) skin protectant:w/...,https://www.gpo.gov/fdsys/pkg/CFR-2018-title21...,Possibly Marketed Outside US 1970,Ala Quin by Crown Laboratories,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,SUNBURN,Approved
3,Possibly Marketed Outside US,ALPHA-KETOGLUTARATE,http://naturaldatabase.therapeuticresearch.com...,NaN,NaN,NaN,Osteoporosis; Gastrointestinal microbiota imba...,Not Provided; Not Provided; Not Provided; Appr...
4,US Approved OTC,21 CFR 344.12 otic:ear drying aid isopropyl al...,https://www.gpo.gov/fdsys/pkg/CFR-2018-title21...,US Previously Marketed 1921,Rubbing Alcohol,https://books.google.com/books?id=1n4gAQAAMAAJ...,,
...,...,...,...,...,...,...,...,...
2316,US Approved Rx 2018,NDA207078,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 2018,NDA207078,https://www.accessdata.fda.gov/scripts/cder/da...,,
2317,US Approved Rx 2013,ANDA091363,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 2001,ZOMETA by NOVARTIS,https://www.accessdata.fda.gov/scripts/cder/da...,Postmenopausal osteoporosis; Hypercalcemia; Pr...,Approved; Approved; Approved; Approved; Approved
2318,US Approved Rx 2016,ANDA205074,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1997,ZOMIG by IPR,https://www.accessdata.fda.gov/scripts/cder/da...,Migraine with aura; Migraine without aura,Approved; Approved
2319,US Approved Rx 2016,ANDA201509,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 1992,NDA019908,https://www.accessdata.fda.gov/scripts/cder/da...,Insomnia,Approved


In [32]:
links_df = pd.DataFrame(links, columns=['CAS', 'URL'])
links_df

,CAS,URL
0,623-84-7,https://drugs.ncats.io/drug/5Z492UNF9O
1,36653-82-4,https://drugs.ncats.io/drug/936JST6JCN
2,112-92-5,https://drugs.ncats.io/drug/2KR89I4H1Y
3,328-50-7,https://drugs.ncats.io/drug/8ID597Z82X
4,67-63-0,https://drugs.ncats.io/drug/ND2M416302
...,...,...
2316,17141-74-1,https://drugs.ncats.io/drug/D652ZWF066
2317,118072-93-8,https://drugs.ncats.io/drug/70HZ18PH24
2318,139264-17-8,https://drugs.ncats.io/drug/2FS66TH3YW
2319,82626-48-0,https://drugs.ncats.io/drug/7K383OQI23


In [33]:
result = pd.concat([links_df, merged], axis=1)
result

,CAS,URL,Highest Development Event Status and Year,Highest Development Event Source ID,Highest Development Event Source URL,Earliest Approved Event Status and Year,Earliest Approved Event Source ID,Earliest Approved Event Source URL,conditions,phases
0,623-84-7,https://drugs.ncats.io/drug/5Z492UNF9O,Possibly Marketed Outside US 1996,ANDA040166,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,Possibly Marketed Outside US 1996,ANDA040166,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,Bacillus cereus infection,Basic research
1,36653-82-4,https://drugs.ncats.io/drug/936JST6JCN,US Previously Marketed 1990,EXOSURF NEONATAL by GLAXOSMITHKLINE,https://www.accessdata.fda.gov/scripts/cder/da...,Possibly Marketed Outside US 1984,NU-DERM SUNFADER Skin Lightener with Sunscree...,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,"Dry, itchy skin; Respiratory distress syndrome...",Approved; Approved
2,112-92-5,https://drugs.ncats.io/drug/2KR89I4H1Y,US Previously Marketed,21 CFR 310.545(a)(18)(i)(B) skin protectant:w/...,https://www.gpo.gov/fdsys/pkg/CFR-2018-title21...,Possibly Marketed Outside US 1970,Ala Quin by Crown Laboratories,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,SUNBURN,Approved
3,328-50-7,https://drugs.ncats.io/drug/8ID597Z82X,Possibly Marketed Outside US,ALPHA-KETOGLUTARATE,http://naturaldatabase.therapeuticresearch.com...,NaN,NaN,NaN,Osteoporosis; Gastrointestinal microbiota imba...,Not Provided; Not Provided; Not Provided; Appr...
4,67-63-0,https://drugs.ncats.io/drug/ND2M416302,US Approved OTC,21 CFR 344.12 otic:ear drying aid isopropyl al...,https://www.gpo.gov/fdsys/pkg/CFR-2018-title21...,US Previously Marketed 1921,Rubbing Alcohol,https://books.google.com/books?id=1n4gAQAAMAAJ...,,
...,...,...,...,...,...,...,...,...,...,...
2316,17141-74-1,https://drugs.ncats.io/drug/D652ZWF066,US Approved Rx 2018,NDA207078,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 2018,NDA207078,https://www.accessdata.fda.gov/scripts/cder/da...,,
2317,118072-93-8,https://drugs.ncats.io/drug/70HZ18PH24,US Approved Rx 2013,ANDA091363,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 2001,ZOMETA by NOVARTIS,https://www.accessdata.fda.gov/scripts/cder/da...,Postmenopausal osteoporosis; Hypercalcemia; Pr...,Approved; Approved; Approved; Approved; Approved
2318,139264-17-8,https://drugs.ncats.io/drug/2FS66TH3YW,US Approved Rx 2016,ANDA205074,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1997,ZOMIG by IPR,https://www.accessdata.fda.gov/scripts/cder/da...,Migraine with aura; Migraine without aura,Approved; Approved
2319,82626-48-0,https://drugs.ncats.io/drug/7K383OQI23,US Approved Rx 2016,ANDA201509,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 1992,NDA019908,https://www.accessdata.fda.gov/scripts/cder/da...,Insomnia,Approved


In [34]:
result.tail(10)

,CAS,URL,Highest Development Event Status and Year,Highest Development Event Source ID,Highest Development Event Source URL,Earliest Approved Event Status and Year,Earliest Approved Event Source ID,Earliest Approved Event Source URL,conditions,phases
2311,30516-87-1,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2312,111406-87-2,https://drugs.ncats.io/drug/V1L22WVE2S,US Approved Rx 2020,ANDA211390,https://www.accessdata.fda.gov/scripts/cder/da...,Possibly Marketed Outside US 1996,NDA020471,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,Asthma,Approved
2313,7440-66-6,https://drugs.ncats.io/drug/J41CSQ7QDS,US Approved OTC,21 CFR 333.110(a) first aid antibiotic:ointmen...,https://www.gpo.gov/fdsys/pkg/CFR-2018-title21...,US Previously Marketed 1921,Zinc U.S.P.,https://www.gutenberg.org/files/41778/41778-h/...,Skin diseases; Infection; Superficial infectio...,Approved; Approved; Approved
2314,146939-27-7,https://drugs.ncats.io/drug/6UKA5VEJ6X,US Approved Rx 2012,ANDA077565,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 2001,NDA020825,https://www.accessdata.fda.gov/scripts/cder/da...,Schizophrenia; Bipolar disorder,Approved; Approved
2315,7440-67-7,https://drugs.ncats.io/drug/C6V6S92N3C,US Previously Marketed,21 CFR 310.502(a) certain drugs zirconium,https://www.gpo.gov/fdsys/pkg/CFR-2018-title21...,NaN,NaN,NaN,,
2316,17141-74-1,https://drugs.ncats.io/drug/D652ZWF066,US Approved Rx 2018,NDA207078,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 2018,NDA207078,https://www.accessdata.fda.gov/scripts/cder/da...,,
2317,118072-93-8,https://drugs.ncats.io/drug/70HZ18PH24,US Approved Rx 2013,ANDA091363,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 2001,ZOMETA by NOVARTIS,https://www.accessdata.fda.gov/scripts/cder/da...,Postmenopausal osteoporosis; Hypercalcemia; Pr...,Approved; Approved; Approved; Approved; Approved
2318,139264-17-8,https://drugs.ncats.io/drug/2FS66TH3YW,US Approved Rx 2016,ANDA205074,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1997,ZOMIG by IPR,https://www.accessdata.fda.gov/scripts/cder/da...,Migraine with aura; Migraine without aura,Approved; Approved
2319,82626-48-0,https://drugs.ncats.io/drug/7K383OQI23,US Approved Rx 2016,ANDA201509,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 1992,NDA019908,https://www.accessdata.fda.gov/scripts/cder/da...,Insomnia,Approved
2320,68291-97-4,https://drugs.ncats.io/drug/459384H98V,US Approved Rx 2006,ANDA077634,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 2000,NDA020789,https://www.accessdata.fda.gov/scripts/cder/da...,Epilepsy,Approved


In [35]:
result.to_csv("inxight_data_1.csv")